# Import Libraries

In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
import nest_asyncio
nest_asyncio.apply()

# Make sure LLM and Embedding Models are present

In [ ]:
import ollama
ollama.pull('llama3')
ollama.pull('nomic-embed-text')
base_url='http://localhost:11434'

In [ ]:
Settings.llm = Ollama(model='llama3',request_timeout=3600,base_url=base_url,temperature=0)
Settings.embed_model = OllamaEmbedding(model_name='nomic-embed-text',base_url=base_url)

# Get the tools for document from utils

In [ ]:
from utils import get_doc_tools
import os

In [ ]:
base_path = os.getcwd()
file_name = 'attention_is_all_you_need.pdf'

In [ ]:
abs_path = os.path.join(base_path,file_name)

In [ ]:
vector_tool, summary_tool = get_doc_tools(abs_path)

# Build Agent

```Llama3, Phi3 does not support Function Calling API. Hence we are using ReActAgent```

In [ ]:
from llama_index.core.agent import ReActAgent

In [ ]:
agent = ReActAgent.from_tools([vector_tool,summary_tool], llm=Settings.llm, verbose=True,)

# Check the Deafult Prompt and make changes if required

In [ ]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")
    print("------------------------------------------------------------------------------")

In [ ]:
from llama_index.core import PromptTemplate

react_system_header_str = """\

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools
You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}

## Output Format
To answer the question, please use the following format.

```
Thought: I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "query"}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do this {{"input": "query"}}.

If this format is used, the user will respond in the following format:

```
Observation: tool response
```

You should keep repeating the above format until you have enough information
to answer the question without using any more tools. At that point, you MUST respond
in the one of the following two formats:

```
Thought: I can answer without using any more tools.
Answer: [your answer here]
```

```
Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
```

## Additional Rules
- The answer MUST contain a sequence of bullet points that explain how you arrived at the answer. This can include aspects of the previous conversation history.
- You MUST obey the function signature of each tool. Do NOT pass in no arguments if the function expects arguments.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.

"""
react_system_prompt = PromptTemplate(react_system_header_str)

In [ ]:
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [ ]:
agent.get_prompts()

# Perfrom Actions with AGENT

In [ ]:
response = agent.query(
    "Tell me about multi head attention in transformer."
)

In [ ]:
response = agent.query(
    "Tell me about multi head attention in transformer and how is it different fron simple sttention?"
)

# Agent Reasoning with Memory

In [ ]:
response = agent.stream_chat(
    "Tell me about the evaluation datasets used."
)

In [ ]:
response_gen = response.response_gen

for token in response_gen:
    print(token, end="", flush=True)

In [ ]:
response = agent.stream_chat("Tell me the results over one of the above datasets.")

In [ ]:
response_gen = response.response_gen

for token in response_gen:
    print(token, end="")